In [69]:
import wfdb
import numpy as np
import pygame

pygame.init()
font = pygame.font.Font(None, 36)

def get_color(value):
    value = min(value, 1)
    value = max(value, 0)
    # Example color mapping function
    return int(value * 255), 0, 255 - int(value * 255)

def viz_emg(screen, emg_frame):
    left = 50
    top = 50
    scale = 30
    size = 1
    
    for i in range(4):
        for j in range(8):
            for k in range(8):
                pygame.draw.rect(screen, get_color(emg_frame[i*64+j*k]),
                                 (left+(i//2)*scale*10+scale*j,
                                  top+(i%2)*scale*10+scale*k,
                                  scale*size,
                                  scale*size))

In [70]:
# Read a WFDB record
record = wfdb.rdrecord('data/dynamic_preprocess_sample1')

# Display basic information about the record
print(record.__dict__)

{'record_name': 'dynamic_preprocess_sample1', 'n_sig': 256, 'fs': 2048, 'counter_freq': None, 'base_counter': None, 'sig_len': 2048, 'base_time': None, 'base_date': None, 'comments': [], 'sig_name': ['ED-8-8', 'ED-8-7', 'ED-8-6', 'ED-8-5', 'ED-8-4', 'ED-8-3', 'ED-8-2', 'ED-8-1', 'ED-7-8', 'ED-7-7', 'ED-7-6', 'ED-7-5', 'ED-7-4', 'ED-7-3', 'ED-7-2', 'ED-7-1', 'ED-6-8', 'ED-6-7', 'ED-6-6', 'ED-6-5', 'ED-6-4', 'ED-6-3', 'ED-6-2', 'ED-6-1', 'ED-5-8', 'ED-5-7', 'ED-5-6', 'ED-5-5', 'ED-5-4', 'ED-5-3', 'ED-5-2', 'ED-5-1', 'ED-4-8', 'ED-4-7', 'ED-4-6', 'ED-4-5', 'ED-4-4', 'ED-4-3', 'ED-4-2', 'ED-4-1', 'ED-3-8', 'ED-3-7', 'ED-3-6', 'ED-3-5', 'ED-3-4', 'ED-3-3', 'ED-3-2', 'ED-3-1', 'ED-2-8', 'ED-2-7', 'ED-2-6', 'ED-2-5', 'ED-2-4', 'ED-2-3', 'ED-2-2', 'ED-2-1', 'ED-1-8', 'ED-1-7', 'ED-1-6', 'ED-1-5', 'ED-1-4', 'ED-1-3', 'ED-1-2', 'ED-1-1', 'EP-8-8', 'EP-8-7', 'EP-8-6', 'EP-8-5', 'EP-8-4', 'EP-8-3', 'EP-8-2', 'EP-8-1', 'EP-7-8', 'EP-7-7', 'EP-7-6', 'EP-7-5', 'EP-7-4', 'EP-7-3', 'EP-7-2', 'EP-7-1', 

In [71]:
emg_raw = np.array(record.__dict__['p_signal'].T)
num_frames = emg_raw.shape[-1]

print("emg.shape:"+str(emg_raw.shape))

emg.shape:(256, 2048)


In [72]:
(np.min(emg_raw, keepdims=True), np.max(emg_raw, keepdims=True))

(array([[-4.93815172]]), array([[3.49807233]]))

In [73]:
emg_hdsmg = emg_raw
#emg_hdsmg = emg_hdsmg - np.mean(emg_hdsmg[:, 0:10], axis=1, keepdims=True)

emg_hdsmg_min = np.min(emg_hdsmg, axis=1, keepdims=True)
emg_hdsmg_max = np.max(emg_hdsmg, axis=1, keepdims=True)
emg_hdsmg_range = emg_hdsmg_max - emg_hdsmg_min
emg_hdsmg = (emg_hdsmg - emg_hdsmg_min) / emg_hdsmg_range

In [74]:
(np.min(emg_hdsmg, keepdims=True), np.max(emg_hdsmg, keepdims=True))

(array([[0.]]), array([[1.]]))

In [75]:
fps = 300
class EmgKinVisualizer:
    def __init__(self, width=800, height=600):
        self.speed = 1
        self.width = width
        self.height = height
        self.current_frame = 0
        self.running = False

        self.screen = pygame.display.set_mode(
            (width, height + 50))  # Extra height for timeline
        pygame.display.set_caption("EMG and Kinematics Visualizer")

        self.timeline_rect = pygame.Rect(50, height + 10, width - 100, 30)
        self.slider_rect = pygame.Rect(50, height + 10, 10, 30)
        self.exit_button_rect = pygame.Rect(700, 10, 60, 30)

    def run(self):
        clock = pygame.time.Clock()

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    return
                elif event.type == pygame.KEYDOWN:
                    self.handle_key_event(event.key)
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    if event.button == 1:  # Left mouse button
                        if self.timeline_rect.collidepoint(event.pos):
                            self.update_frame_from_mouse(event.pos[0])
                        if self.exit_button_rect.collidepoint(event.pos):
                            pygame.quit()
                            return
                elif event.type == pygame.MOUSEMOTION:
                    if event.buttons[0]:  # Left mouse button held down
                        if self.timeline_rect.collidepoint(event.pos):
                            self.update_frame_from_mouse(event.pos[0])

            if self.running:
                self.current_frame += self.speed
                if self.current_frame >= num_frames:
                    self.current_frame = 0
                elif self.current_frame < 0:
                    self.current_frame = num_frames - 1

            self.draw()
            pygame.display.flip()
            clock.tick(fps/3)

    def draw(self):
        self.screen.fill((0, 0, 0))
        
        # Draw emg data
        viz_emg(self.screen, emg_hdsmg[:, self.current_frame])

        # Draw timeline
        pygame.draw.rect(self.screen, (100, 100, 100), self.timeline_rect)
        slider_x = int(self.timeline_rect.left +
                       (self.current_frame / num_frames) * self.timeline_rect.width)
        self.slider_rect.left = slider_x
        pygame.draw.rect(self.screen, (200, 200, 200), self.slider_rect)

        # Draw current frame number
        text = font.render(
            f"Frame: {self.current_frame}/{num_frames-1}", True, (200, 200, 200))
        self.screen.blit(text, (10, self.height - 30))

        # Draw exit button
        pygame.draw.rect(self.screen, (200, 50, 50), self.exit_button_rect)
        exit_text = font.render("Exit", True, (255, 255, 255))
        self.screen.blit(exit_text, (self.exit_button_rect.x + 10, self.exit_button_rect.y + 5))

    def handle_key_event(self, key):
        if key == pygame.K_SPACE:
            self.running = not self.running
        elif key == pygame.K_RIGHT:
            self.speed = 1
            self.running = True
        elif key == pygame.K_LEFT:
            self.speed = -1
            self.running = True
        elif key == pygame.K_UP:
            self.speed = min(self.speed * 2, 16)
        elif key == pygame.K_DOWN:
            self.speed = max(self.speed // 2, 1)

    def update_frame_from_mouse(self, mouse_x):
        timeline_x = mouse_x - self.timeline_rect.left
        frame_ratio = timeline_x / self.timeline_rect.width
        self.current_frame = int(frame_ratio * num_frames)
        self.current_frame = max(0, min(self.current_frame, num_frames - 1))
        self.running = False


if __name__ == "__main__":
    visualizer = EmgKinVisualizer()
    visualizer.run()